# 第1章 pytorchのきほん

## 1.2 Tensor
* Tensorはnumpyのndarrayとほぼ同様のAPI + GPUを利用できる
* torch.Tensorは、torch.FloatTensorのエイリアスっぽい？

In [1]:
import numpy as np
import torch
import random

In [4]:
# seedの固定
# torchのシード固定は、実行直前に毎回セットしないとだめっぽい（randomとかでもそうだっけ...？）
seed_value = 72
# np.random.seed(seed_value)
# random.seed(seed_value)
# torch.manual_seed(seed_value)
# torch.cuda.manual_seed(seed_value)

In [4]:
# Tensorの生成
t = torch.Tensor([[1,2], [3,4]])

In [5]:
t

tensor([[1., 2.],
        [3., 4.]])

In [6]:
# GPUにTensorを作成する
# WSLだとGPUが認識できないっぽい
t2 = torch.cuda.FloatTensor([[1,2], [3,4]])

In [7]:
t2

tensor([[1., 2.],
        [3., 4.]], device='cuda:0')

In [8]:
# ndarrayを渡し、倍精度のTensorを作る
x = np.array([[1,2], [3,4]])
t3 = torch.DoubleTensor(x)

In [9]:
t3

tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64)

In [10]:
# 0-9の数値で初期化された1次元のTensor
#t4 = torch.arange(0, 10) # どちらでもいい
t4 = torch.Tensor(np.arange(0,10))
t4

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [11]:
# すべての値が0の100x10のTensorを作成し、cudaメソッドでGPUに転送する
t5 = torch.zeros(100,10).cuda()

In [12]:
# 正規乱数で100x10のTensorを生成
t6 = torch.randn(100,10)

In [13]:
# Tensorのshapeはsizeメソッドで取得可能
# shapeでも可
print(t6.size())
print(t6.shape)

torch.Size([100, 10])
torch.Size([100, 10])


In [14]:
# numpyメソッドを使用してndarrayに変換
t7 = torch.Tensor([[1, 2], [3, 4]])
x7 = t.numpy()
x7

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [15]:
# GPU上のTensorはcpuメソッドで一度CPUのtensorに変換しないとndarrayにはできない
t8 = t7.cuda()
x8 = t8.cpu().numpy()
x8

array([[1., 2.],
       [3., 4.]], dtype=float32)

In [16]:
# Tensorは基本的にndarrayと同様のインデックス操作が可能
t9 = torch.Tensor([[1, 2, 3], [4, 5, 6]])

print(t9)
print(t9[0, 2])
print(t9[:, :2])
print(t9[:, [1,2]])
print(t9[t9 > 3])
t9[0, 1] = 100
print(t9)
t9[:, 1] = 200
print(t9)
t9[t9 > 10] = 20
print(t9)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor(3.)
tensor([[1., 2.],
        [4., 5.]])
tensor([[2., 3.],
        [5., 6.]])
tensor([4., 5., 6.])
tensor([[  1., 100.,   3.],
        [  4.,   5.,   6.]])
tensor([[  1., 200.,   3.],
        [  4., 200.,   6.]])
tensor([[ 1., 20.,  3.],
        [ 4., 20.,  6.]])


In [17]:
# 四則演算はTensor同士、またはTensorとスカラーでのみ可能
# Tensorとndarrayは混在できない
# Tensor同士でも型は一致させる必要がある
# ndarray同様に、Tensornにもブロードキャスト（次元の補完）が適用される

v = torch.Tensor([1, 2, 3])
w = torch.Tensor([0, 10, 20])
m = torch.Tensor([[0, 1, 2], [10, 200, 300]])

v2 = v + 10
print(v2)
v2 = v ** 2
print(v2)

z = v - w
print(z)

u = 2 * v - w / 10 + 6.0
print(u)

m2 = m * 2.0
print(m2)

m3 = m + v
print(m3)

m4 = m + m
print(m4)

tensor([11., 12., 13.])
tensor([1., 4., 9.])
tensor([  1.,  -8., -17.])
tensor([ 8.,  9., 10.])
tensor([[  0.,   2.,   4.],
        [ 20., 400., 600.]])
tensor([[  1.,   3.,   5.],
        [ 11., 202., 303.]])
tensor([[  0.,   2.,   4.],
        [ 20., 400., 600.]])


In [18]:
# 各種関数も使える
torch.manual_seed(seed_value) # seed固定
X = torch.randn(100, 10)

# 絶対値
y = X * 2 + torch.abs(X)
# 平均
m = torch.mean(X)
print(m)
# 関数ではなくメソッドとしても使える
m = m.mean()
print(m)
# 集計は次元を指定できる
m2 = X.mean(axis = 0)
print(m2)

tensor(0.0206)
tensor(0.0206)
tensor([-0.0733,  0.1366,  0.1345,  0.0627,  0.0248, -0.0725,  0.0262,  0.1065,
        -0.0265, -0.1127])


In [22]:
# 次元の操作
x1 = torch.Tensor([[1, 2], [3, 4]]) # 2x2
x2 = torch.Tensor([[10, 20, 30], [40, 50, 60]])
print(x1)
print(x2)

# 2x2を4x1に見せる
print(x1.view(4, 1))
# -1は残りの次元を表し、一度だけ使用できる
# 以下の場合、最初に1を指定したので残りは4
print(x1.view(1, -1))

# 転置
print(x2.t())

# dim=1に対して結合することで、2x5のTensorを作る
print(torch.cat([x1, x2], dim=1))

# HWCをCWHに変換
# 64x32x3のデータが100個
torch.manual_seed(seed_value)
hwc_img_data = torch.rand(100, 64, 32, 3)
print(hwc_img_data.shape)
chw_img_data = hwc_img_data.transpose(1, 2).transpose(1, 3)
print(chw_img_data.shape)

tensor([[1., 2.],
        [3., 4.]])
tensor([[10., 20., 30.],
        [40., 50., 60.]])
tensor([[1.],
        [2.],
        [3.],
        [4.]])
tensor([[1., 2., 3., 4.]])
tensor([[10., 40.],
        [20., 50.],
        [30., 60.]])
tensor([[ 1.,  2., 10., 20., 30.],
        [ 3.,  4., 40., 50., 60.]])
torch.Size([100, 64, 32, 3])
torch.Size([100, 3, 64, 32])


In [26]:
# 行列の演算
torch.manual_seed(seed_value)
m = torch.randn(100, 10)
torch.manual_seed(seed_value)
v = torch.randn(10)
print(m.shape)
print(v.shape)

# 内積
d = torch.dot(v, v)
print(d)

# 行列とベクトルの積
v2 = torch.mv(m, v)
print(v2.shape)

# 行列積
m2 = torch.mm(m.t(), m)
print(m2.shape)

# 特異値分解
u, s, v = torch.svd(m)
print(u.shape)
print(s)
print(v.shape)

torch.Size([100, 10])
torch.Size([10])
tensor(16.4310)
torch.Size([100])
torch.Size([10, 10])
torch.Size([100, 10])
tensor([13.0095, 12.0607, 11.4284, 11.1951, 10.2369,  9.3851,  9.0325,  8.5119,
         8.2606,  7.3641])
torch.Size([10, 10])


## 1.3 Variableと自動微分

**どうやらVariableはpytorch0.4からTensorに統合されたらしく、Tensorをそのままbackwardできるようになったもよう**

https://qiita.com/vintersnow/items/91545c27e2003f62ebc4

**Variableはdeprecatedなので、使わないほうがよい**

https://pytorch.org/docs/stable/autograd.html#variable-deprecated

* variableはTensorを拡張し、自動微分を扱えるようにしたもの
* NNではパラメータやデータは全てVariableを使用する
* Tensorの演算はそのまま利用できるが、TensorとVariableを混ぜて演算することはできない
* VariableのdataプロパティにアクセスするとTensorを取り出せる
* Variableに対して演算を積み重ねると計算グラフが構築され、backwardメソッドを呼ぶと自動的に微分を計算できる
* NNの最適化なんかで重要らしい（わからん）

In [2]:
from torch.autograd import Variable as V

In [27]:
torch.manual_seed(seed_value)
_x = V(torch.randn(100, 3))
print(_x[:3])
# Variableは使わなくてよい
torch.manual_seed(seed_value)
x = torch.randn(100, 3)
print(x[:3])
print(x.dtype)

# 微分の変数として扱う場合はrequires_gradフラグをTrueにする
_a = V(torch.Tensor([1, 2, 3]), requires_grad=True)
print(_a)
# Variableを使わない書き方
# torch.Tensorではなくtorch.tensorを使い、dtypeとrequires_gradを指定する
# 渡すリストの中身をfloatにしておけばdtypeを指定しなくてもよい
# a = torch.tensor([1, 2, 3], dtype=torch.float32, requires_grad=True)
a = torch.tensor([1., 2., 3.], requires_grad=True)
print(a)
print(a.dtype)

# 計算をすることで自動的に計算グラフが構築される
# ここでy = xia1 + yia2 + zia3がi=1～i=100まで作られる
y = torch.mv(x, a)
print(y.shape)
print(y)

# ここでy = a1(x1 + x2 + x3 + ... + x100) + a2(y1 + y2 + y3 + ... + y100) + a1(z1 + z2 + z3 + ... + z100)
o = y.sum()
print(o)

# 微分を実行
o.backward()

tensor([[-0.6095,  0.0248, -0.6509],
        [-1.2133, -0.9720,  0.5561],
        [-0.8389, -2.5504, -0.6438]])
tensor([[-0.6095,  0.0248, -0.6509],
        [-1.2133, -0.9720,  0.5561],
        [-0.8389, -2.5504, -0.6438]])
torch.float32
tensor([1., 2., 3.], requires_grad=True)
tensor([1., 2., 3.], requires_grad=True)
torch.float32
torch.Size([100])
tensor([-2.5125, -1.4888, -7.8712,  0.7108,  3.9587, -4.7207, -5.9617, -3.2727,
        -1.1687,  1.6212,  0.8945,  5.2887, -1.2928, -4.7624, -5.4628,  3.4641,
        -5.0585, -0.9466, -0.8145, -1.9913, -8.6125, -0.5781, -5.9557,  2.3592,
         7.0098,  1.0560,  1.0235, -6.0381, -8.7797,  3.3768,  0.4492, -7.9153,
         7.5558,  6.8641, -2.9943,  1.3821, -4.1092,  1.5527,  0.5417,  0.4636,
         0.8153, -3.9358,  0.3027, -2.1342, -1.7281,  4.5834, -1.4723, -3.1393,
         3.5540,  0.9848,  4.4618, -5.5899,  1.4705,  1.7438,  1.8758, -1.2955,
        -5.4876, 10.5755, -5.9045, -6.7668, -0.5909, -2.8809,  0.7169,  7.2164,
        

In [25]:
# 解析解と比較
print(a.grad) # oをaで微分するので、xi, yi, ziのsumが残る
print(x.sum(0))
print(x.grad is None) # xはrequires_gradがFalseなので微分は計算されない

tensor([-11.4051,   4.9093, -12.2099])
tensor([-11.4051,   4.9093, -12.2099])
True


In [48]:
# もうちょい単純な例（下記より）
# https://qiita.com/mananam/items/f4be3fb0d996a6a3eae3
# テンソルを作成
# requires_grad=Trueで自動微分対象を指定
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)

# 計算グラフを構築
# y = 2 * x + 3
y = w * x + b
print(y)

# 勾配を計算
y.backward()

# 勾配を表示
print(x.grad)  # dy/dx = w = 2
print(w.grad)  # dy/dw = x = 1
print(b.grad)  # dy/db = 1

tensor(5., grad_fn=<AddBackward0>)
tensor(2.)
tensor(1.)
tensor(1.)
